# EDA for 'debiaser' data product
#### Sagar Setru, September 16th, 2020

## Brief description using CoNVO framework

### Context

Some people are eager to get news from outside of their echo chamber. However, they do not know where to go outside of their echo chambers, and may also have some activation energy when it comes to seeking information from other sources. In the meantime, most newsfeeds only push you content that you agree with. You end up in an echo chamber, but may not have ever wanted to be in one in the first place.

### Need

A way to find news articles from different yet reliable media sources.

### Vision

Debiaser, a data product (maybe Chrome plug-in?) that will recommend news articles similar in topic to the one currently being read, but from several pre-curated and reliable news media organizations across the political spectrum, for example, following the "media bias chart" here https://www.adfontesmedia.com/ or the "media bias ratings" here: https://www.allsides.com/media-bias/media-bias-ratings. The app will determine the main topics of the text of a news article, and then show links to similar articles from other news organizations.

Caveats: Many of these articles may be behind paywalls. News aggregators already basically do this. How different is this than just searching Google using the title of an article?

### Outcome

People who are motivated to engage in content outside of their echo chambers have a tool that enables them to quickly find news similar to what they are currently reading, but from a variety of news organizations.

# EDA

In [35]:
# make sure I'm in the right environment

print('Conda environment:')
print(os.environ['CONDA_DEFAULT_ENV'])

Conda environment:
insight


In [36]:
# import base packages

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [240]:
# import text processing and NLP specific packages

# for generating LDA models
import gensim

# to break articles up into sentences (currently not in use)
from nltk import tokenize

In [262]:
def load_stop_words_csv_to_list(full_file_name):
    """fxn that loads stop words list downloaded from git repo called 'news-stopwords'"""
    
    stop_words = pd.read_csv(full_file_name)

    stop_words = stop_words['term']

    stop_words = [word for word in stop_words]
    
    return stop_words

In [38]:
# get list of manually made text files of articles

full_path = '/Users/sagarsetru/Documents/post PhD positions search/insightDataScience/project/debiaser/article_text_files/'

full_file_names = [
full_path+'ap_hurricane_sally_unleashes_20200916.txt',
full_path+'cnn_big_ten_backtracks_20200916.txt',
full_path+'nyt_on_the_fire_line_20200915.txt',
]

In [286]:
# choose list of stop words

# choose whether 1k, 10k, 100k, or nltk
which_stop_words = '1k'
# which_stop_words = '10k'
# which_stop_words = '100k'
# which_stop_words = 'nltk'

stop_words_path = '/Users/sagarsetru/Documents/post PhD positions search/insightDataScience/project/debiaser/stop_words_db/news-stopwords-master/'


if which_stop_words == '1k':
    
    # doing 1k words list
    stop_words_file_name = 'sw1k.csv'
    
    # make full file name
    stop_words_full_file_name = stop_words_path+stop_words_file_name
    
    # get list of stop words
    stop_words = load_stop_words_csv_to_list(stop_words_full_file_name)
    
elif which_stop_words == '10k':
    
    # doing 10k words list
    stop_words_file_name = 'sw10k.csv'

    # make full file name
    stop_words_full_file_name = stop_words_path+stop_words_file_name
    
    # get list of stop words
    stop_words = load_stop_words_csv_to_list(stop_words_full_file_name)

elif which_stop_words == '100k':
    
    # doing 100k
    stop_words_file_name = 'sw100k.csv'  
    
    # get full file name
    stop_words_full_file_name = stop_words_path+stop_words_file_name
    
    # get list of stop words
    stop_words = load_stop_words_csv_to_list(stop_words_full_file_name)


elif which_stop_words == 'nltk':
    # import from nltk
    from nltk.corpus import stopwords
    
    stop_words = stopwords.words('english')
    
else:
    print('Select proper variable name for "which_stop_words"')

In [224]:
# loop through files

# get full file names
for ind, full_file_name in enumerate(full_file_names):
    
    # get the article text as one string, remove new lines
    with open(full_file_name, 'r') as file:
        article_text = file.read().replace('\n', ' ')

    # break article into sentences
    article_sentences = tokenize.sent_tokenize(article_text)
    
    if ind == 1:
        break
        


In [287]:
# Create a set of frequent meaningless words to remove

word_frequency_threshold = 2

# following https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#sphx-glr-auto-examples-core-run-core-concepts-py

# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stop_words]
         for document in [article_text]]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > word_frequency_threshold] for text in texts]

processed_dictionary = Dictionary(processed_corpus)
print(processed_dictionary)

Dictionary(6 unique tokens: ['coaches,', 'college', 'football', 'sports', 'student-athletes']...)


In [289]:
# print(stop_words)

In [290]:
processed_corpus[0][0:20]

['ten',
 'football',
 'ten',
 'college',
 'football',
 'ten',
 'football',
 'ten',
 'coaches,',
 'ten',
 'football',
 'coaches,',
 'coaches,',
 'student-athletes',
 'ten',
 'student-athletes',
 'college',
 'football',
 'ten',
 'sports']

In [291]:
bow_corpus = [processed_dictionary.doc2bow(text) for text in processed_corpus]
lda_2 = gensim.models.LdaModel(corpus=bow_corpus,num_topics=5,id2word=processed_dictionary)

In [292]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_numeric

# # lda.top_topics(corpus=bow_corpus,dictionary=processed_dictionary)
# for i in range(0, lda.num_topics-1):
#     current_topic = lda.print_topic(i)
#     print(current_topic)
    
lda_topics = lda_2.show_topics()

topics = []
filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]

for topic in lda_topics:
#     print(topic)
    topics.append(preprocess_string(topic[1], filters))

all_topic_words = []
for topic in topics:
    print(topic)
    for topic_word in topic:
        print(topic_word)
        all_topic_words.append(topic_word)
    
    print(' ')

print(all_topic_words)

['ten', 'football', 'sports', 'student', 'athletes', 'coaches', 'college']
ten
football
sports
student
athletes
coaches
college
 
['football', 'ten', 'sports', 'coaches', 'college', 'student', 'athletes']
football
ten
sports
coaches
college
student
athletes
 
['ten', 'sports', 'coaches', 'football', 'college', 'student', 'athletes']
ten
sports
coaches
football
college
student
athletes
 
['ten', 'football', 'sports', 'student', 'athletes', 'coaches', 'college']
ten
football
sports
student
athletes
coaches
college
 
['ten', 'football', 'sports', 'college', 'student', 'athletes', 'coaches']
ten
football
sports
college
student
athletes
coaches
 
['ten', 'football', 'sports', 'student', 'athletes', 'coaches', 'college', 'football', 'ten', 'sports', 'coaches', 'college', 'student', 'athletes', 'ten', 'sports', 'coaches', 'football', 'college', 'student', 'athletes', 'ten', 'football', 'sports', 'student', 'athletes', 'coaches', 'college', 'ten', 'football', 'sports', 'college', 'student', 'a